# Trabalho de IA - Parte 2: Análise Exploratória (EDA)

Dataset: Olist E-Commerce

Objetivo: Entender os 8 arquivos CSV e uni-los em um único DataFrame "achatado" para o pré-processamento.
"

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define o caminho (assumindo que o notebook está em /notebooks)
DATA_PATH = "../data/raw/"
print(f"Arquivos na pasta: {os.listdir(DATA_PATH)}")

Arquivos na pasta: ['olist_customers_dataset.csv', 'olist_geolocation_dataset.csv', 'olist_orders_dataset.csv', 'olist_order_items_dataset.csv', 'olist_order_payments_dataset.csv', 'olist_order_reviews_dataset.csv', 'olist_products_dataset.csv', 'olist_sellers_dataset.csv', 'product_category_name_translation.csv']


In [2]:
# Carregar os principais CSVs
try:
    customers = pd.read_csv(DATA_PATH + "olist_customers_dataset.csv")
    orders = pd.read_csv(DATA_PATH + "olist_orders_dataset.csv")
    items = pd.read_csv(DATA_PATH + "olist_order_items_dataset.csv")
    products = pd.read_csv(DATA_PATH + "olist_products_dataset.csv")
    payments = pd.read_csv(DATA_PATH + "olist_order_payments_dataset.csv")
    
    print("Arquivos CSV carregados com sucesso.")
except FileNotFoundError as e:
    print(f"Erro ao carregar CSV: {e}")
    print("Verifique se você baixou e descompactou os dados na pasta 'data/raw/'.")

Arquivos CSV carregados com sucesso.


"""
## O "Trabalho de Mais": Unindo as Tabelas

O Olist é um banco de dados relacional. Precisamos fazer 'joins' (merges) para criar um dataset único.

- `orders` junta com `customers` (via `customer_id`)
- `orders` junta com `items` (via `order_id`)
- `items` junta com `products` (via `product_id`)
"""

In [3]:
# 1. Juntar pedidos com clientes
df = pd.merge(
    orders, customers, on="customer_id", how="left"
)

# 2. Juntar os itens de cada pedido (pode criar duplicatas de pedidos, normal)
df = pd.merge(
    df, items, on="order_id", how="left"
)

# 3. Juntar os produtos de cada item
df = pd.merge(
    df, products, on="product_id", how="left"
)

# 4. Juntar os pagamentos (opcional, mas útil)
df = pd.merge(
    df, payments, on="order_id", how="left"
)

print(f"Shape do dataset 'achatado': {df.shape}")
df.head()

Shape do dataset 'achatado': (118434, 30)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,3.0,voucher,1.0,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,268.0,4.0,500.0,19.0,8.0,13.0,2.0,voucher,1.0,18.59
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,178.0,1.0,400.0,19.0,13.0,19.0,1.0,boleto,1.0,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,232.0,1.0,420.0,24.0,19.0,21.0,1.0,credit_card,3.0,179.12


In [4]:
# Verificar os tipos de dados e valores nulos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118434 entries, 0 to 118433
Data columns (total 30 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       118434 non-null  object 
 1   customer_id                    118434 non-null  object 
 2   order_status                   118434 non-null  object 
 3   order_purchase_timestamp       118434 non-null  object 
 4   order_approved_at              118258 non-null  object 
 5   order_delivered_carrier_date   116360 non-null  object 
 6   order_delivered_customer_date  115037 non-null  object 
 7   order_estimated_delivery_date  118434 non-null  object 
 8   customer_unique_id             118434 non-null  object 
 9   customer_zip_code_prefix       118434 non-null  int64  
 10  customer_city                  118434 non-null  object 
 11  customer_state                 118434 non-null  object 
 12  order_item_id                 